<a href="https://colab.research.google.com/github/Swathi-Thandavarayan/LLM-Chatbot/blob/main/LLM_Based_Chatbot_with_RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#LLM-Based Chatbot with RAG Pipeline

In [2]:
!pip install openai spacy faiss-cpu streamlit python-dotenv
!python -m spacy download en_core_web_md


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 54.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
OPENAI_API_KEY=your_openai_key_here

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")


In [ ]:
import spacy
import faiss
import numpy as np

class Retriever:
    def __init__(self, filepath):
        self.nlp = spacy.load("en_core_web_md")
        self.texts = self._load_docs(filepath)
        self.embeddings = np.array([self.nlp(text).vector for text in self.texts]).astype("float32")
        self.index = self._build_faiss_index(self.embeddings)

    def _load_docs(self, filepath):
        with open(filepath, 'r') as f:
            return [line.strip() for line in f if line.strip()]

    def _build_faiss_index(self, vectors):
        index = faiss.IndexFlatL2(vectors.shape[1])
        index.add(vectors)
        return index

    def query(self, user_input, k=3):
        user_vector = self.nlp(user_input).vector.astype("float32").reshape(1, -1)
        distances, indices = self.index.search(user_vector, k)
        return [self.texts[i] for i in indices[0]]


In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import openai
import streamlit as st
from retriever import Retriever
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load retriever
retriever = Retriever("data/sample_docs.txt")

def generate_response(query, context):
    prompt = f"Context:\n{context}\n\nUser Question: {query}\n\nAnswer:"
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.7,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Streamlit UI
st.title("LLM Chatbot with RAG")
user_query = st.text_input("Ask something:")

if user_query:
    context = retriever.query(user_query, k=3)
    full_context = "\n".join(context)
    answer = generate_response(user_query, full_context)

    st.subheader("Context Retrieved:")
    for i, ctx in enumerate(context, 1):
        st.markdown(f"**Doc {i}:** {ctx}")

    st.subheader("Generated Answer:")
    st.write(answer)
